# Calculadora de Separacion Flash Multicomponente
### CoolProp HEOS (GERG-2008)
---
> **Instrucciones:** Ejecuta en orden: Celda 1 (instalar) -> Celda 2 (clases) -> Celda 3 (interfaz interactiva).

> **Zona bifasica tipica (gas natural):** T=150-240 K, P=5-30 bar

In [1]:
# CELDA 1: Instalacion de dependencias
import subprocess, sys
print('Instalando CoolProp...')
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'CoolProp', '-q'])
print('Listo')

Instalando CoolProp...
Listo


In [2]:
# CELDA 2: Clases (ConfigManagerInMemory, FlashModel, PlotService)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import CoolProp.CoolProp as CP
from CoolProp.CoolProp import AbstractState
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

# PALETA
STYLE = {
    'bg':'#0F1923','panel':'#162030','border':'#1E3048',
    'text':'#E8EEF4','subtext':'#7A9BB5','accent':'#00D4FF',
    'vapor':'#00D4FF','liquid':'#FF6B35','feed':'#A8D8A8',
    'crit':'#FFD700','grid':'#1A2E42',
}
COMP_COLORS=['#00D4FF','#FF6B35','#A8D8A8','#FFD700','#C77DFF',
             '#FF9F1C','#2EC4B6','#E71D36','#F72585','#4CC9F0','#7B2FBE']

def _apply_style(fig, axes):
    fig.patch.set_facecolor(STYLE['bg'])
    for ax in axes:
        ax.set_facecolor(STYLE['panel'])
        ax.tick_params(colors=STYLE['subtext'], labelsize=8)
        ax.xaxis.label.set_color(STYLE['text'])
        ax.yaxis.label.set_color(STYLE['text'])
        ax.title.set_color(STYLE['accent'])
        for s in ax.spines.values(): s.set_edgecolor(STYLE['border'])
        ax.grid(True, color=STYLE['grid'], linewidth=0.6, linestyle='--', alpha=0.7)

class ConfigManagerInMemory:
    def __init__(self, mezcla, simulacion):
        self._config = {'mezcla': mezcla, 'simulacion': simulacion}
    def get_full_config(self):    return self._config
    def get_flash_config(self):   return self._config.get('simulacion', {})
    def get_mixture_config(self): return self._config.get('mezcla', [])

class FlashModel:
    def __init__(self, temperature, pressure, config_manager):
        self.T = temperature; self.P = pressure
        self.config_manager = config_manager
        self.mezcla_datos = config_manager.get_mixture_config()
        self.components = [c['coolprop_name']    for c in self.mezcla_datos]
        self.composition = [c['z_fraccion_molar'] for c in self.mezcla_datos]
        self.mixture_code = '&'.join(self.components)
        self.state = AbstractState('HEOS', self.mixture_code)
        self.state.set_mole_fractions(self.composition)
        self.state.update(CP.PT_INPUTS, self.P, self.T)
        self.phase = self.state.phase()
        self.phase_names = {
            CP.iphase_liquid:'Liquido', CP.iphase_gas:'Vapor',
            CP.iphase_supercritical:'Supercritico',
            CP.iphase_supercritical_gas:'Gas supercritico',
            CP.iphase_supercritical_liquid:'Liquido supercritico',
            CP.iphase_twophase:'Bifasico (VLE)',
            CP.iphase_unknown:'Desconocido',
        }
    def get_phase_name(self):
        return self.phase_names.get(self.phase, f'Codigo {self.phase}')

class PlotService:
    def __init__(self, fm):
        self.fm = fm; self._extract()

    def _extract(self):
        fm = self.fm; self.n = len(fm.components)
        try: self.labels = [c.get('id_espanol', c['coolprop_name']) for c in fm.mezcla_datos]
        except: self.labels = fm.components
        self.z = np.array(fm.composition)
        self.is_twophase = (fm.phase == CP.iphase_twophase)
        if self.is_twophase:
            self.beta  = fm.state.Q()
            self.x_liq = np.array(fm.state.mole_fractions_liquid())
            self.y_vap = np.array(fm.state.mole_fractions_vapor())
            self.K     = np.where(self.x_liq > 1e-15, self.y_vap / self.x_liq, np.nan)
        else:
            self.beta  = 1.0 if fm.phase == CP.iphase_gas else 0.0
            self.x_liq = self.z.copy(); self.y_vap = self.z.copy(); self.K = np.ones(self.n)
        try:
            sim = fm.config_manager.get_flash_config()
            self.F_total = sim['feed']['flow_kmol_h']
        except: self.F_total = 100.0
        self.V_flow = self.beta * self.F_total
        self.L_flow = (1 - self.beta) * self.F_total
        self.F_comp = self.z * self.F_total
        self.V_comp = self.y_vap * self.V_flow
        self.L_comp = self.x_liq * self.L_flow

    def dashboard(self, save_path=None):
        fig = plt.figure(figsize=(18, 11))
        fig.patch.set_facecolor(STYLE['bg'])
        gs = gridspec.GridSpec(2, 3, figure=fig, hspace=0.45, wspace=0.35,
                               left=0.06, right=0.97, top=0.88, bottom=0.07)
        axes = [fig.add_subplot(gs[r, c]) for r in range(2) for c in range(3)]
        _apply_style(fig, axes)
        self._envelope(axes[0]); self._comps(axes[1]); self._kvals(axes[2])
        self._flows(axes[3]);    self._dist(axes[4]);   self._table(axes[5])
        fase = 'BIFASICO (VLE)' if self.is_twophase else 'UNA FASE'
        fig.suptitle(
            f'REPORTE FLASH  T={self.fm.T:.1f} K  P={self.fm.P/1e5:.2f} bar  {fase}  b={self.beta:.4f}',
            fontsize=12, fontweight='bold', color=STYLE['accent'], y=0.95)
        if save_path: plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor=STYLE['bg'])
        plt.show()

    def _envelope(self, ax):
        T_arr = np.linspace(120, 400, 70)
        Tb, Pb, Td, Pd = [], [], [], []
        for T in T_arr:
            for Q, Tl, Pl in [(0, Tb, Pb), (1, Td, Pd)]:
                try:
                    AS = AbstractState('HEOS', self.fm.mixture_code)
                    AS.set_mole_fractions(self.fm.composition)
                    AS.update(CP.QT_INPUTS, Q, T)
                    p = AS.p()
                    if 1e3 < p < 1.5e8: Tl.append(T); Pl.append(p / 1e6)
                except: pass
        if Tb: ax.plot(Tb, Pb, color=STYLE['liquid'], lw=2, label='Burbuja (Q=0)')
        if Td: ax.plot(Td, Pd, color=STYLE['vapor'],  lw=2, label='Rocio (Q=1)')
        if Tb and Td: ax.fill(Tb + Td[::-1], Pb + Pd[::-1], alpha=0.12, color=STYLE['accent'])
        ax.scatter([self.fm.T], [self.fm.P / 1e6], s=140, zorder=10,
                   color=STYLE['crit'], edgecolors='white', linewidths=1.5,
                   label=f'Op. ({self.fm.T:.0f}K, {self.fm.P/1e6:.2f}MPa)')
        ax.set_xlabel('Temperatura (K)'); ax.set_ylabel('Presion (MPa)')
        ax.set_title('Envolvente de Fases')
        ax.legend(fontsize=7, facecolor=STYLE['panel'], labelcolor=STYLE['text'], edgecolor=STYLE['border'])

    def _comps(self, ax):
        x = np.arange(self.n); w = 0.25
        for offset, data, color, lbl in [
            (-w, self.z,     STYLE['feed'],   'z (feed)'),
            ( 0, self.x_liq, STYLE['liquid'], 'x (liq)'),
            ( w, self.y_vap, STYLE['vapor'],  'y (vap)'),
        ]:
            bars = ax.bar(x + offset, data, w, color=color, alpha=0.85,
                          edgecolor=STYLE['bg'], label=lbl)
            for b, h in zip(bars, data):
                if h > 0.005:
                    ax.text(b.get_x() + b.get_width()/2, h + 0.005, f'{h:.3f}',
                            ha='center', va='bottom', fontsize=6, color=STYLE['subtext'])
        ax.set_xticks(x)
        ax.set_xticklabels(self.labels, rotation=25, ha='right', fontsize=8)
        ax.set_ylabel('Fraccion molar')
        ax.set_ylim(0, min(1.0, max(self.z.max(), self.x_liq.max(), self.y_vap.max()) * 1.2))
        ax.set_title('Composiciones por Fase')
        ax.legend(fontsize=7, facecolor=STYLE['panel'], labelcolor=STYLE['text'], edgecolor=STYLE['border'])

    def _kvals(self, ax):
        x = np.arange(self.n)
        Ks = np.where(np.isnan(self.K), 1e-6, self.K)
        cols = [STYLE['vapor'] if k >= 1 else STYLE['liquid'] for k in Ks]
        bars = ax.bar(x, Ks, color=cols, alpha=0.85, edgecolor=STYLE['bg'])
        ax.axhline(1, color=STYLE['crit'], lw=1.5, ls='--', alpha=0.8)
        for b, k in zip(bars, Ks):
            ax.text(b.get_x() + b.get_width()/2, b.get_height() * 1.05, f'{k:.3f}',
                    ha='center', va='bottom', fontsize=6.5, color=STYLE['subtext'])
        ax.set_yscale('log'); ax.set_xticks(x)
        ax.set_xticklabels(self.labels, rotation=25, ha='right', fontsize=8)
        ax.set_ylabel('K = y/x  (log)'); ax.set_title('Factores de Equilibrio K')
        ax.legend(handles=[
            mpatches.Patch(color=STYLE['vapor'],  label='K>1 vapor'),
            mpatches.Patch(color=STYLE['liquid'], label='K<1 liquido'),
        ], fontsize=7, facecolor=STYLE['panel'], labelcolor=STYLE['text'], edgecolor=STYLE['border'])

    def _flows(self, ax):
        x = np.arange(self.n); w = 0.3
        ax.bar(x - w/2, self.V_comp, w, color=STYLE['vapor'],  alpha=0.85,
               edgecolor=STYLE['bg'], label=f'Vapor V={self.V_flow:.1f} kmol/h')
        ax.bar(x + w/2, self.L_comp, w, color=STYLE['liquid'], alpha=0.85,
               edgecolor=STYLE['bg'], label=f'Liquido L={self.L_flow:.1f} kmol/h')
        ax.scatter(x, self.F_comp, s=60, zorder=10, color=STYLE['feed'],
                   edgecolors='white', linewidths=1, label=f'Feed F={self.F_total:.1f} kmol/h')
        ax.set_xticks(x); ax.set_xticklabels(self.labels, rotation=25, ha='right', fontsize=8)
        ax.set_ylabel('Flujo molar (kmol/h)'); ax.set_title('Balance de Materia')
        ax.legend(fontsize=7, facecolor=STYLE['panel'], labelcolor=STYLE['text'], edgecolor=STYLE['border'])

    def _dist(self, ax):
        fV = self.V_comp / self.F_total; fL = self.L_comp / self.F_total
        bv = bl = 0.0
        for i in range(self.n):
            c = COMP_COLORS[i % len(COMP_COLORS)]
            ax.bar(0, fV[i], bottom=bv, width=0.4, color=c, alpha=0.85, edgecolor=STYLE['bg'])
            ax.bar(1, fL[i], bottom=bl, width=0.4, color=c, alpha=0.85, edgecolor=STYLE['bg'])
            if fV[i] > 0.02:
                ax.text(0, bv + fV[i]/2, f'{self.labels[i][:5]}\n{fV[i]:.2f}',
                        ha='center', va='center', fontsize=6, color='white', fontweight='bold')
            if fL[i] > 0.02:
                ax.text(1, bl + fL[i]/2, f'{self.labels[i][:5]}\n{fL[i]:.2f}',
                        ha='center', va='center', fontsize=6, color='white', fontweight='bold')
            bv += fV[i]; bl += fL[i]
        ax.set_xticks([0, 1])
        ax.set_xticklabels([f'VAPOR\nb={self.beta:.3f}', f'LIQUIDO\n1-b={1-self.beta:.3f}'],
                           fontsize=9, color=STYLE['text'])
        ax.set_ylabel('Fraccion del feed'); ax.set_ylim(0, 1.05)
        ax.set_title('Distribucion de Fases')

    def _table(self, ax):
        ax.axis('off')
        cols = ['Componente','z','x','y','K','F\n(kmol/h)','L\n(kmol/h)','V\n(kmol/h)']
        rows = [[self.labels[i], f'{self.z[i]:.4f}', f'{self.x_liq[i]:.4f}',
                 f'{self.y_vap[i]:.4f}',
                 f'{self.K[i]:.4f}' if not np.isnan(self.K[i]) else '-',
                 f'{self.F_comp[i]:.2f}', f'{self.L_comp[i]:.2f}', f'{self.V_comp[i]:.2f}']
                for i in range(self.n)]
        rows.append(['TOTAL', f'{sum(self.z):.4f}', f'{sum(self.x_liq):.4f}',
                     f'{sum(self.y_vap):.4f}', '-',
                     f'{self.F_total:.2f}', f'{self.L_flow:.2f}', f'{self.V_flow:.2f}'])
        tbl = ax.table(cellText=rows, colLabels=cols, loc='center', cellLoc='center')
        tbl.auto_set_font_size(False); tbl.set_fontsize(7); tbl.scale(1, 1.45)
        for (r, c), cell in tbl.get_celld().items():
            cell.set_facecolor(STYLE['panel']); cell.set_edgecolor(STYLE['border'])
            if r == 0:
                cell.set_facecolor(STYLE['border'])
                cell.set_text_props(color=STYLE['accent'], fontweight='bold')
            elif r == len(rows):
                cell.set_facecolor('#1A3050')
                cell.set_text_props(color=STYLE['crit'], fontweight='bold')
            else:
                cell.set_text_props(color=STYLE['text'])
        ax.set_title('Tabla de Resultados', color=STYLE['accent'], pad=10)

    def export_csv(self, path='flash_resultados.csv'):
        df = pd.DataFrame({
            'Componente': self.labels, 'z': self.z,
            'x_liquido': self.x_liq,  'y_vapor': self.y_vap,
            'K': self.K, 'F_kmolh': self.F_comp,
            'L_kmolh': self.L_comp,   'V_kmolh': self.V_comp,
        })
        df.to_csv(path, index=False)
        print(f'Exportado: {path}')
        return df

print('Clases cargadas correctamente')


Clases cargadas correctamente


In [ ]:
# CELDA 3: Interfaz interactiva

GERG_COMPONENTS = {
    'Metano':'Methane','Etano':'Ethane','Propano':'Propane',
    'n-Butano':'n-Butane','Isobutano':'IsoButane','n-Pentano':'n-Pentane',
    'Isopentano':'Isopentane','n-Hexano':'n-Hexane','n-Heptano':'n-Heptane',
    'n-Octano':'n-Octane','Nitrogeno':'Nitrogen','CO2':'CO2',
    'H2S':'H2S','Hidrogeno':'Hydrogen','Agua':'Water','Decano':'n-Decane',
}

w_T = widgets.FloatSlider(value=200,min=120,max=400,step=1,description='T (K):',
    style={'description_width':'70px'},layout=widgets.Layout(width='500px'),
    continuous_update=False)
w_P = widgets.FloatSlider(value=20,min=1,max=100,step=0.5,description='P (bar):',
    style={'description_width':'70px'},layout=widgets.Layout(width='500px'),
    continuous_update=False)
w_F = widgets.FloatText(value=100.0,description='Feed (kmol/h):',
    style={'description_width':'110px'},layout=widgets.Layout(width='250px'))

MAX_COMP = 8
defaults = [('Metano',0.50),('Etano',0.20),('Propano',0.15),('n-Butano',0.10),('Nitrogeno',0.05)]
comp_rows = []
for i in range(MAX_COMP):
    active = i < len(defaults)
    esp    = defaults[i][0] if active else list(GERG_COMPONENTS.keys())[i]
    zv     = defaults[i][1] if active else 0.0
    chk = widgets.Checkbox(value=active,description='',
                           layout=widgets.Layout(width='30px'),indent=False)
    drp = widgets.Dropdown(options=list(GERG_COMPONENTS.keys()),value=esp,
                           layout=widgets.Layout(width='165px'),disabled=not active)
    sld = widgets.FloatText(value=zv,description='z:',
                            style={'description_width':'20px'},
                            layout=widgets.Layout(width='130px'),disabled=not active)
    def toggle(change, d=drp, s=sld):
        d.disabled = not change['new']; s.disabled = not change['new']
    chk.observe(toggle, names='value')
    row = widgets.HBox([chk,drp,sld],layout=widgets.Layout(align_items='center',margin='2px 0'))
    comp_rows.append({'row':row,'chk':chk,'drp':drp,'sld':sld})

suma_lbl = widgets.HTML(value='')
def update_suma(*a):
    total = sum(r['sld'].value for r in comp_rows if r['chk'].value)
    color = '#4ECDC4' if abs(total-1)<1e-6 else ('#FFD700' if abs(total-1)<0.05 else '#FF6B6B')
    icon  = 'OK' if abs(total-1)<1e-6 else ('!' if abs(total-1)<0.05 else 'X')
    suma_lbl.value = f"<span style='color:{color};font-weight:bold;font-size:13px'>[{icon}] Sz = {total:.4f}</span>"
for r in comp_rows:
    r['sld'].observe(update_suma, names='value')
    r['chk'].observe(update_suma, names='value')
update_suma()

btn_run  = widgets.Button(description='CALCULAR FLASH',
    layout=widgets.Layout(width='200px',height='40px'),
    style={'button_color':'#00D4FF','font_weight':'bold'})
btn_csv  = widgets.Button(description='Exportar CSV',
    layout=widgets.Layout(width='155px',height='40px'),
    style={'button_color':'#1E3048'},disabled=True)
btn_norm = widgets.Button(description='Normalizar z',
    layout=widgets.Layout(width='155px',height='40px'),
    style={'button_color':'#1E3048'})
status_lbl = widgets.HTML(value='')
out = widgets.Output()
state = {'ps': None}

def on_norm(b):
    activos = [r for r in comp_rows if r['chk'].value]
    tot = sum(r['sld'].value for r in activos)
    if tot > 0:
        for r in activos: r['sld'].value = round(r['sld'].value / tot, 6)
btn_norm.on_click(on_norm)

def on_run(b):
    with out:
        clear_output(wait=True)
        activos = [r for r in comp_rows if r['chk'].value]
        tot = sum(r['sld'].value for r in activos)
        if abs(tot - 1) > 0.01:
            status_lbl.value = f"<span style='color:#FF6B6B'>Error: Sz={tot:.4f} - debe ser 1.00. Usa Normalizar z.</span>"
            return
        if len(activos) < 2:
            status_lbl.value = "<span style='color:#FF6B6B'>Error: necesitas al menos 2 componentes.</span>"
            return
        status_lbl.value = "<span style='color:#FFD700'>Calculando...</span>"
        mezcla = [{'id_espanol':r['drp'].value,
                   'coolprop_name':GERG_COMPONENTS[r['drp'].value],
                   'z_fraccion_molar':round(r['sld'].value, 6)} for r in activos]
        sim = {'feed':{'flow_kmol_h':w_F.value},
               'flash':{'temperature_K':w_T.value,'pressure_Pa':w_P.value*1e5}}
        try:
            cfm = ConfigManagerInMemory(mezcla, sim)
            fm  = FlashModel(w_T.value, w_P.value*1e5, cfm)
            ps  = PlotService(fm); state['ps'] = ps
            fase = fm.get_phase_name()
            color = '#4ECDC4' if fm.phase == CP.iphase_twophase else '#FFD700'
            status_lbl.value = (f"<span style='color:{color};font-weight:bold;font-size:13px'>"
                f'OK  Fase: {fase}  |  b={ps.beta:.4f}  |  '
                f'V={ps.V_flow:.1f} kmol/h  |  L={ps.L_flow:.1f} kmol/h</span>')
            btn_csv.disabled = False; ps.dashboard()
        except Exception as e:
            status_lbl.value = f"<span style='color:#FF6B6B'>Error: {e}</span>"
btn_run.on_click(on_run)

def on_csv(b):
    if state['ps']:
        df = state['ps'].export_csv('flash_resultados.csv')
        with out:
            display(df.style
                .set_properties(**{'background-color':'#162030','color':'#E8EEF4'})
                .format(precision=4))
btn_csv.on_click(on_csv)

# Ensamblar UI
display(widgets.HTML(
    "<div style='background:linear-gradient(135deg,#0F1923,#162030);"
    "border:1px solid #1E3048;border-radius:8px;padding:16px 20px;margin-bottom:12px'>"
    "<h2 style='color:#00D4FF;font-family:monospace;margin:0 0 4px 0'>Flash Multicomponente</h2>"
    "<p style='color:#7A9BB5;margin:0;font-size:12px'>CoolProp HEOS - GERG-2008</p></div>"
))
display(widgets.HTML("<h3 style='color:#00D4FF;font-family:monospace'>Condiciones Operativas</h3>"))
display(w_T); display(w_P); display(w_F)
display(widgets.HTML(
    "<h3 style='color:#00D4FF;font-family:monospace;margin-top:12px'>Composicion de la Mezcla</h3>"
    "<p style='color:#7A9BB5;font-size:12px;margin:0'>Solo componentes GERG-2008</p>"
))
display(widgets.HTML(
    "<div style='display:flex;gap:20px;color:#7A9BB5;font-size:11px;"
    "font-family:monospace;padding-left:35px'>"
    "<span style='width:165px'>Componente</span><span>Fraccion molar z</span></div>"
))
for r in comp_rows: display(r['row'])
display(widgets.HBox([suma_lbl], layout=widgets.Layout(margin='6px 0 4px 35px')))
display(widgets.HBox([btn_run, btn_norm, btn_csv],
                     layout=widgets.Layout(margin='12px 0', gap='10px')))
display(status_lbl)
display(out)


HTML(value="<div style='background:linear-gradient(135deg,#0F1923,#162030);border:1px solid #1E3048;border-rad…

HTML(value="<h3 style='color:#00D4FF;font-family:monospace'>Condiciones Operativas</h3>")

FloatSlider(value=200.0, continuous_update=False, description='T (K):', layout=Layout(width='500px'), max=400.…

FloatSlider(value=20.0, continuous_update=False, description='P (bar):', layout=Layout(width='500px'), min=1.0…

FloatText(value=100.0, description='Feed (kmol/h):', layout=Layout(width='250px'), style=DescriptionStyle(desc…

HTML(value="<h3 style='color:#00D4FF;font-family:monospace;margin-top:12px'>Composicion de la Mezcla</h3><p st…

HTML(value="<div style='display:flex;gap:20px;color:#7A9BB5;font-size:11px;font-family:monospace;padding-left:…

HTML(value='')

Output()

---
## Guia de uso

| Paso | Accion |
|------|--------|
| 1 | Ajusta **T** y **P** con los sliders |
| 2 | Activa componentes con el checkbox izquierdo |
| 3 | Ingresa fraccion molar **z** para cada componente activo |
| 4 | Usa **Normalizar z** si la suma no es exactamente 1.00 |
| 5 | Presiona **CALCULAR FLASH** |
| 6 | Exporta resultados con **Exportar CSV** |

## Componentes disponibles (GERG-2008)
Metano, Etano, Propano, n-Butano, Isobutano, n-Pentano, Isopentano,
n-Hexano, n-Heptano, n-Octano, Nitrogeno, CO2, H2S, Hidrogeno, Agua, Decano
